In [0]:
topChannels = spark.read.parquet('dbfs:/FileStore/refined/topChannels')
topChannelsMetrics = spark.read.parquet('dbfs:/FileStore/refined/topChannelsMetrics')
topVideos = spark.read.parquet('dbfs:/FileStore/refined/topVideos')
topVideosMetrics = spark.read.parquet('dbfs:/FileStore/refined/topVideosMetrics')
mostSubscribed = spark.read.parquet('dbfs:/FileStore/refined/mostSubscribed')
mostSubscribedMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostSubscribedMetrics')
mostPopularChannels = spark.read.parquet('dbfs:/FileStore/refined/mostPopularChannels')
mostPopularChannelsMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostPopularChannelsMetrics')
mostPopularVideos = spark.read.parquet('dbfs:/FileStore/refined/mostPopularVideos')
mostPopularVideosMetrics = spark.read.parquet('dbfs:/FileStore/refined/mostPopularVideosMetrics')


In [0]:
print('topChannels')
display(topChannels)
print('topChannelsMetrics')
display(topChannelsMetrics)
print('topVideos')
display(topVideos)
print('topVideosMetrics')
display(topVideosMetrics)
print('mostSubscribed')
display(mostSubscribed)
print('mostSubscribedMetrics')
display(mostSubscribedMetrics)
print('mostPopularChannels')
display(mostPopularChannels)
print('mostPopularChannelsMetrics')
display(mostPopularChannelsMetrics)
print('mostPopularVideos')
display(mostPopularVideos)
print('mostPopularVideosMetrics')
display(mostPopularVideosMetrics)

topChannels


rank,youtuber,subscribers,video_views,video_count,category,started
626,little mix,16200000,7323194490,278,music,2011
627,reikvevo,3510000,7315774447,225,music,2009
628,djsnakevevo,6510000,7310124942,58,music,2014
629,lospolinesios,25600000,7308727483,942,people & blogs,2012
630,themermaidscales,7110000,7307742113,2031,entertainment,2017
631,pica - pica oficial,10000000,7305496502,286,music,2013
632,oddbods - official channel,9860000,7302179681,1420,film & animation,2014
633,troom troom es,19400000,7300617134,2737,people & blogs,2016
634,shopee indonesia,2630000,7300217491,4734,people & blogs,2015
635,a for adley - learning & fun,5240000,7299054283,599,entertainment,2017


topChannelsMetrics


rank,youtuber,subscribers/year,video_views/year,video_count/year
626,little mix,1350000,610266207,23
627,reikvevo,250714,522555317,16
628,djsnakevevo,723333,812236104,6
629,lospolinesios,2327272,664429771,85
630,themermaidscales,1185000,1217957018,338
631,pica - pica oficial,1000000,730549650,28
632,oddbods - official channel,1095555,811353297,157
633,troom troom es,2771428,1042945304,391
634,shopee indonesia,328750,912527186,591
635,a for adley - learning & fun,873333,1216509047,99


topVideos


...

rank,youtuber,subscribers/year,video_views/year,video_count/year
501,finatticzvevo,8409,4935026,0
502,paloma mami,174166,55061241,1
503,anselelgortvevo,48714,6403665,0
504,twenty one pilots,935714,356196772,13
505,鈴原るる【にじさんじ所属】,131250,690888,0
506,cjzz17,52000,22528597,0
507,sebastianyatravevo,311111,935762332,12
508,davidrafaelafon,6426,3991880,0
509,markronsonvevo,261428,402777321,8
510,sam,3160,1479478,0


mostPopularVideos


rank,video,video_views,likes,dislikes,category,published
376,пронько. экономика: набиуллина ответила на вопросы «царьграда»,1987411,10901,243,news & politics,2017
377,emkal - oublie-moi (clip officiel),10903494,213459,null,music,2022
378,releasing a stranded octopus and it thanked me,12029468,112065,3130,pets & animals,2013
379,patrick wasserziehr lässt seinen kaffee umrühren,1265446,9952,7681,people & blogs,2017
380,zapowiedź pociągu,643936,7279,384,people & blogs,2018
381,counting to jay-z and back,544904,6743,281,music,2017
382,bacilos la cara de la luna,2142731,2971,89,music,2008
383,bird karma,1324926,24519,843,people & blogs,2017
384,in flames - trigger,8393871,30145,709,music,2006
385,"the room - 'i did not hit her!' - 'oh, hi mark!' - 🍆 see film 👑the diana clone 🌹on amazon 👸",2571193,20280,636,comedy,2016


mostPopularVideosMetrics


rank,video,video_views/year,likes/year,dislikes/year
376,пронько. экономика: набиуллина ответила на вопросы «царьграда»,331235,1816,40
377,emkal - oublie-moi (clip officiel),10903494,213459,0
378,releasing a stranded octopus and it thanked me,1202946,11206,313
379,patrick wasserziehr lässt seinen kaffee umrühren,210907,1658,1280
380,zapowiedź pociągu,128787,1455,76
381,counting to jay-z and back,90817,1123,46
382,bacilos la cara de la luna,142848,198,5
383,bird karma,220821,4086,140
384,in flames - trigger,493757,1773,41
385,"the room - 'i did not hit her!' - 'oh, hi mark!' - 🍆 see film 👑the diana clone 🌹on amazon 👸",367313,2897,90


## Top canais que tbm estão na lista de mais inscritos

In [0]:
display(topChannels.join(mostSubscribed.select('youtuber'), topChannels.youtuber == mostSubscribed.youtuber, 'inner').orderBy('rank'))

rank,youtuber,subscribers,video_views,video_count,category,started,youtuber
1,t-series,236000000,215448719927,18742,music,2006,t-series
2,cocomelon - nursery rhymes,154000000,151495855157,857,education,2006,cocomelon - nursery rhymes
3,set india,151000000,139508515842,104998,shows,2006,set india
4,sony sab,77200000,92411723276,64568,shows,2007,sony sab
5,✿ kids diana show,108000000,88008748808,1065,people & blogs,2015,✿ kids diana show
6,like nastya,104000000,87845862611,760,people & blogs,2016,like nastya
7,wwe,93400000,74213926197,66653,sports,2007,wwe
8,vlad and niki,93400000,72957816965,527,entertainment,2018,vlad and niki
9,movieclips,58700000,59058009915,39988,film & animation,2006,movieclips
10,colors tv,59900000,57721177728,103569,shows,2008,colors tv


## Top canais que não estao na lista de mais inscritos

In [0]:
display(topChannels.join(mostSubscribed.select('youtuber'), topChannels.youtuber == mostSubscribed.youtuber, 'left_anti').orderBy('rank'))

rank,youtuber,subscribers,video_views,video_count,category,started
101,inside edition,11500000,20591639620,29111,news & politics,2012
148,mbcentertainment,8930000,16668532647,95793,entertainment,2009
154,theweekndvevo,10500000,16093052559,136,music,2011
172,fischer's-フィッシャーズ-,7870000,15457323338,3029,entertainment,2012
182,family fun pack,10100000,15009857285,2393,people & blogs,2011
187,monilina,10700000,14819324503,739,comedy,2021
218,fox news,10300000,13765558200,89515,news & politics,2006
220,asianet,8750000,13728670903,108514,entertainment,2008
225,tarang tv,6990000,13529383890,44418,entertainment,2012
230,karolgvevo,3720000,13443895582,66,music,2016


## Canis mais incritos que não são Top canais

In [0]:
display(mostSubscribed.join(topChannels.select('youtuber'), topChannels.youtuber == mostSubscribed.youtuber, 'left_anti').orderBy('rank'))

rank,youtuber,subscribers,video_views,video_count,category,started
2,youtube movies,162000000,0,0,film & animation,2015
6,music,118000000,0,0,null,2013
10,gaming,93300000,0,0,null,2013
18,sports,75100000,0,0,null,2013
57,whinderssonnunes,44000000,4187535166,524,comedy,2013
59,holasoygerman.,43500000,4790314608,136,entertainment,2011
71,kimberly loaiza,39700000,5009678822,252,people & blogs,2016
81,carryminati,37900000,3144815534,182,comedy,2014
92,news,36400000,0,0,null,2013
102,youtube,35200000,2951744370,618,news & politics,1970
